In [8]:
import os

import pandas as pd
from pandas import DataFrame
from io import StringIO
import geohash2
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, request_influxdb, query_tangaras, query_data, to_be_checked, is_corr_ok, df_to_csv

## Tangara Sensors

In [9]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-09-26T00:00:00-05:00 -> 1695704400000, Until: 2023-09-26T00:05:00-05:00 -> 1695704700000


In [10]:
# Query Tangaras
influxdb_sql_query_tangaras = query_tangaras(start_timestamp, end_timestamp)

print(influxdb_sql_query_tangaras)

SELECT DISTINCT(geo) AS "geohash" FROM "fixed_stations_01" WHERE ("geo3" = 'd29') AND time >= 1695704400000ms AND time <= 1695704700000ms GROUP BY "name";


In [11]:
def df_tangaras(start_timestamp: int, end_timestamp: int) -> DataFrame:
    """
    Get Data Frame Tangaras Sensors of all Tangara sensors that have reported data over a period of time.

    :params:
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms

    :return: DataFrame, Tangaras Sensors
    """
    # Query Tangaras
    influxdb_sql_query_tangaras = query_tangaras(start_timestamp, end_timestamp)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(influxdb_sql_query_tangaras)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame Tangaras
    df_tangaras = pd.read_csv(StringIO(influxdb_request.text), sep=",")

    # Remove/Add Columns
    df_tangaras = df_tangaras[['tags', 'geohash']]
    df_tangaras['MAC'] = df_tangaras['tags'].apply(lambda x: x.split('=')[1])
    df_tangaras['GEOLOCATION'] = df_tangaras['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
    df_tangaras['LATITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    df_tangaras['LONGITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    df_tangaras['tags'] = df_tangaras['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    df_tangaras.rename(columns={'tags': 'ID', 'geohash': 'GEOHASH'}, inplace=True)
    
    # Date time when query is executed
    #tz = timezone(timedelta(hours=-5))
    #df_tangaras['DATETIME'] = datetime.now(tz=tz)

    # Set Index
    df_tangaras.set_index('ID', inplace=True)
    
    return df_tangaras

In [12]:
# Data Frame Tangaras
tangaras = df_tangaras(start_timestamp, end_timestamp)
tangaras.drop_duplicates(subset=['MAC'], inplace=True)

print(f"Period of Time: Since: {START_ISO8601_DATETIME}, Until: {END_ISO8601_DATETIME}")
print(f"Total Tangara Sensors: {len(tangaras)}")

tangaras.head()

Period of Time: Since: 2023-09-26T00:00:00-05:00, Until: 2023-09-26T00:05:00-05:00
Total Tangara Sensors: 12


,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
ID,,,,,
TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.3789825439453125,-76.54106140136719
TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.4613800048828125,-76.51222229003906
TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.4586334228515625,-76.46415710449219
TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.4847259521484375,-76.50260925292969
TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.4517669677734375,-76.55067443847656


In [13]:
# Save Tangaras into CSV file
df_to_csv(tangaras, "tangaras.csv")